In [5]:
df = spark.range(10).toDF("number")

In [6]:
df.select (df["number"]).collect()

[Row(number=0),
 Row(number=1),
 Row(number=2),
 Row(number=3),
 Row(number=4),
 Row(number=5),
 Row(number=6),
 Row(number=7),
 Row(number=8),
 Row(number=9)]

In [7]:
df.select (df["number"]+10).collect()

[Row((number + 10)=10),
 Row((number + 10)=11),
 Row((number + 10)=12),
 Row((number + 10)=13),
 Row((number + 10)=14),
 Row((number + 10)=15),
 Row((number + 10)=16),
 Row((number + 10)=17),
 Row((number + 10)=18),
 Row((number + 10)=19)]

In [8]:
from pyspark.sql.types import *

In [10]:
b= ByteType()

In [11]:
import os

path = os.getcwd()

print(path)

C:\Users\dewair


In [23]:
# read the file 2015-summary.json file, all the file mentioned in the book are uploaded to the guthub repository 
df = spark.read.format("json").load("2015-summary.json")


In [25]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



In [33]:
spark.read.format("json").load("2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [34]:
df.count()

256

In [37]:
#df.show(2)

In [38]:
#you can manipulate columns with col or column function , but columns must be exist within dataframes
# we will stick to use col function 
from pyspark.sql.functions import col, column

In [39]:
col("DEST_COUNTRY_NAME")

Column<b'DEST_COUNTRY_NAME'>

In [40]:
column("count")

Column<b'count'>

In [54]:
# to access the columns of the DF
#val df = spark.read.option("multiline","true").json("C:\\data\\nested-data.json")
#df.select("a.b").show()
#df = spark.read.option("multiline","true").json("2015-summary.json")
df = spark.read.format("json").load("2015-summary.json")

In [55]:
df.select("ORIGIN_COUNTRY_NAME").show()

+-------------------+
|ORIGIN_COUNTRY_NAME|
+-------------------+
|            Romania|
|            Croatia|
|            Ireland|
|      United States|
|              India|
|          Singapore|
|            Grenada|
|      United States|
|      United States|
|      United States|
|       Sint Maarten|
|   Marshall Islands|
|      United States|
|      United States|
|      United States|
|      United States|
|           Paraguay|
|      United States|
|      United States|
|          Gibraltar|
+-------------------+
only showing top 20 rows



In [57]:
# you will see that the datatype exist in the beginning of the rows "the default data type of DF Rows"
df.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [61]:
# you can create ROWS manually by builtin ROW function
from pyspark.sql import Row
myrow = Row("hello" , None , False)

In [63]:
#accessing Row
myrow [0]

'hello'

## Creating  DataFrames
### we can create data frames by Row function , read files, on the fly by taking set of rows and convert them to dataframe

In [76]:
df = spark.read.format("json").load("2015-summary.json")
df.createOrReplaceTempView("dftable")

In [67]:
# on the fly method
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField ,StringType, LongType

In [75]:
mymanulaschema = StructType([
    StructField ("Some", StringType() ,True),
    StructField ("Col" , StringType() , True),
    StructField  ("names" , LongType() , False)
])

In [74]:
myManualSchema = StructType([
  StructField("some", StringType(), True),
  StructField("col", StringType(), True),
  StructField("names", LongType(), False)
])
myRow = Row ("Hello" , None , 1)
MYDF = spark.createDataFrame([myRow] , myManualSchema)

## Select and SelectExpr
#### allow you to do the same thing in the DF

In [78]:
df.select ("ORIGIN_COUNTRY_NAME").show(2)
# in sql ==> select ORIGIN_COUNTRY_NAME from dftable limit 2

+-------------------+
|ORIGIN_COUNTRY_NAME|
+-------------------+
|            Romania|
|            Croatia|
+-------------------+
only showing top 2 rows



In [85]:
# to show the columns in the DF => df.columns
df.select ("ORIGIN_COUNTRY_NAME" , "DEST_COUNTRY_NAME").show(2)


+-------------------+-----------------+
|ORIGIN_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-------------------+-----------------+
|            Romania|    United States|
|            Croatia|    United States|
+-------------------+-----------------+
only showing top 2 rows



In [87]:
#as mentioned before we can refer to columns with [col , column , expr]
from pyspark.sql.functions import col,column, expr
df.select(col("ORIGIN_COUNTRY_NAME"),
          column("ORIGIN_COUNTRY_NAME") , 
          expr("ORIGIN_COUNTRY_NAME")).show(2)

+-------------------+-------------------+-------------------+
|ORIGIN_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-------------------+-------------------+-------------------+
|            Romania|            Romania|            Romania|
|            Croatia|            Croatia|            Croatia|
+-------------------+-------------------+-------------------+
only showing top 2 rows



In [99]:
# expr is the most flexible way to reference columns
df.select (expr ("DEST_COUNTRY_NAME as Distination") ).show(2)
# this in case of change the column name 
df.select (expr("DEST_COUNTRY_NAME as Distination").alias("DEST_COUNTRY_NAME") ).show(2)
# this to change the column name to it's nature 

+-------------+
|  Distination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [95]:
#shorthand for select +expr , spark has provide selectExpr
df.selectExpr ("DEST_COUNTRY_NAME as NewColumnName","DEST_COUNTRY_NAME").show(2)
# the above code rename and print the column , and the second print it as itis

+-------------+-----------------+
|NewColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [97]:
df.selectExpr ("*", "DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME as withincountry").show(2)
# sql => select * , DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME as withincountry from dftable limit 2

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withincountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [101]:
df.selectExpr ("avg(count)" , "count(distinct (DEST_COUNTRY_NAME))").show()

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



## if you want to add expilict values to spark , you must add them by lit function

In [104]:
from pyspark.sql.functions import lit
df.select (expr("*") , lit(1).alias("One")).show(2)
#this add new column "One" with constant value 1 

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



## withColumn function

In [106]:
#if you want to rename or adding new columns to the dataset , you must use withcoulmn function
# withcoulmn function takes 2 arguments coulmn name and expression 
df.withColumn("NumberOne" , lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|NumberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [107]:
#another example , set anew column based on Boolean Flag
df.withColumn("Withincountry" , expr("DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Withincountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [109]:
#use withcolumn in renaming the column
df.withColumn("Destination" , expr("DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|  Destination|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|United States|
|    United States|            Croatia|    1|United States|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



## withColumnRenamed function to rename column explicity

In [110]:
df.withColumnRenamed("DEST_COUNTRY_NAME","Destination").show(2)

+-------------+-------------------+-----+
|  Destination|ORIGIN_COUNTRY_NAME|count|
+-------------+-------------------+-----+
|United States|            Romania|   15|
|United States|            Croatia|    1|
+-------------+-------------------+-----+
only showing top 2 rows



In [ ]:
# remove columns in spark  using drop function
# i don't execute this line , too make the file static 
df.drop ("DEST_COUNTRY_NAME").columns
df.drop ("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME")

In [112]:
# casting from one tpye to another .
#convert the count column from int to long 
df.withColumn("count2" , col("count").cast("long")).show(2)

+-----------------+-------------------+-----+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|count2|
+-----------------+-------------------+-----+------+
|    United States|            Romania|   15|    15|
|    United States|            Croatia|    1|     1|
+-----------------+-------------------+-----+------+
only showing top 2 rows



# Filter The dataframe 

In [117]:
# we can filter the dataframe with filter or where function , and we will stick to where function
df.filter ("count < 2 ").show(2)
#df.filter (col("count") <2 ).show(2)
#df.where ("count <2 ").show(2)
# the above line make the exact 

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [127]:
# to make multiple filters , we must chain multiple where clause
df.where(col ("count") <2).where (col("ORIGIN_COUNTRY_NAME") != "Croatia").show(2)
#df.where("count <2").where ("ORIGIN_COUNTRY_NAME != 'Croatia'").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [134]:
#print the distinct values in DF 
df.select ("ORIGIN_COUNTRY_NAME").distinct().count() # 125
df.select ("ORIGIN_COUNTRY_NAME" , "DEST_COUNTRY_NAME").distinct().count() #256

256

In [135]:
df = spark.read.format("json").load("2015-summary.json")
df.createOrReplaceTempView("dftable")

# Random samples , Random Splits
## concatenate and appending rows 

In [140]:
# to sample some records from the DF (with replacement or without)
# Returns a sampled subset of this :class:`DataFrame`.
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

138

In [141]:
df.sample(withReplacement, fraction, seed).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [144]:
# split the data , as it's common in machine learning,(test & traning)
dataframes = df.randomSplit([0.25,0.57], seed)
print (dataframes[0].count() )
print (dataframes[1].count() )

87
169


In [146]:
# dataframes are immutable  , means can't be changed after creation  . to append to them  , you must union
# to use unioun , make sure they have the same schema ,and number of columns 
from pyspark.sql import Row
schema = df.schema
NewRows =  [
    Row("new Country" , "other Country" ,5),
    Row("new Country 2" , "other Country 3" ,1)

]
parralizedrows = spark.sparkContext.parallelize(NewRows)
newDF = spark.createDataFrame (parralizedrows ,schema )

In [150]:
df.union(newDF).where("count = 1").where("ORIGIN_COUNTRY_NAME = 'other Country 3'").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    new Country 2|    other Country 3|    1|
+-----------------+-------------------+-----+



## Sorting Rows

In [152]:
# we can make sorting by  sort and order by functions
df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [154]:
#df.orderBy("count" , "ORIGIN_COUNTRY_NAME").show(5)
df.orderBy(col("count") , col ("ORIGIN_COUNTRY_NAME") ).show(5) # the same function as above 

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Bahrain|    1|
|    United States|           Bulgaria|    1|
|    United States|            Croatia|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [159]:
# to add more explict sorts , ASC Or DSC
from pyspark.sql.functions import desc, asc
df.orderBy (col("count").desc()).show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+
only showing top 5 rows



In [158]:
df.orderBy (col("count").desc() , col("DEST_COUNTRY_NAME").asc()).show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+
only showing top 5 rows



In [160]:
# for optimizayion purposes , you can ort by partitions with "sortWithinPartitions" Function
df = spark.read.format("json").load("2015-summary.json").sortWithinPartitions("count")

In [162]:
# using limits to extract from DF
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
|    United States|          Gibraltar|    1|
+-----------------+-------------------+-----+



In [163]:
df.orderBy(col("count").desc()).limit(5).show()

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+

